In [3]:
import numpy as np
import shelve
from astropy.table import QTable
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import os
import math
import shutil
from IPython.core.debugger import Tracer
import subprocess
import re
%matplotlib inline



In [2]:
dMass_b[0] #sanity check

-4.8342638674052276

In [3]:
# now to calc mutual incl
dInc_b_deg = dInc_b * u.deg
dInc_b_rad = dInc_b_deg.to(u.rad)
dInc_c_deg = dInc_c * u.deg
dInc_c_rad = dInc_c_deg.to(u.rad)
dLongA_b_deg = dLongA_b * u.deg
dLongA_b_rad = dLongA_b_deg.to(u.rad)
dLongA_c_deg = dLongA_c * u.deg
dLongA_c_rad = dLongA_c_deg.to(u.rad)

mut_incl = np.arccos(np.cos(dInc_b_rad)*np.cos(dInc_c_rad)+
                     np.sin(dInc_b_rad)*np.sin(dInc_c_rad)*np.cos(dLongA_c_rad - dLongA_b_rad))

max_idx_mut_incl = np.argmax(mut_incl)
min_idx_mut_incl = np.argmin(mut_incl)
# gives the run witb the greates mutual inclination. 

max_idx = max_idx_mut_incl
min_idx = min_idx_mut_incl
max_idx

3513

In [35]:
#writes input files. 

t = 25000 # number of runs


n = 0
while n <= t-1: # will make n <= x amount of folders; (25000);
    # make strings of values; b.in
    name_idx = '%05i'%n
    mass_str_b = str(dMass_b[n]) 
    radius_str_b = str(dRadius_b[n])
    eqspin_str_b = str(bForceEqSpin_b)
    obl_str_b = str(dObliquity_b)
    radgy_str_b = str(dRadGyra_b)
    ecc_str_b = str(ecc_b[n])
    semi_str_b = str(semi_b[n])
    dInc_str_b = str(dInc_b[n]) # new
    dLongA_str_b = str(dLongA_b[n]) # new
    dArgP_str_b = str(dArgP_b[n]) # new
    bOutputLapl_str_b = str(bOutputLapl_b) # new
    dPrecA_str_b = str(dPrecA_b[n]) # new
    bCalcDynEllip_str_b = str(bCalcDynEllip_b)
    q_str_b = str(dTidalQ_b[n])
    dK2_str_b = str(dK2_b)
    perts_str_b = str(saTidePerts_b)
    outorder_str_b = str(saOutputOrder_b)
    
    #c.in
    mass_str_c = str(dMass_c[n]) 
    radius_str_c = str(dRadius_c[n])
    eqspin_str_c = str(bForceEqSpin_c)
    obl_str_c = str(dObliquity_c)
    radgy_str_c = str(dRadGyra_c)
    ecc_str_c = str(ecc_c[n])
    semi_str_c = str(semi_c[n])
    dInc_str_c = str(dInc_c[n])
    dLongA_str_c = str(dLongA_c[n])
    dArgP_str_c = str(dArgP_c[n])
    q_str_c = str(dTidalQ_c[n])
    dK2_str_c = str(dK2_c)
    perts_str_c = str(saTidePerts_c)
    outorder_str_c = str(saOutputOrder_c)
    
    
    #star.in
    name_star = name_star
    modules_star = modules_star
    strMass_str = str(dMass_star[n])
    strRad_str = str(dRadius_star[n])
    strObl_str = str(dObliquity_star)
    #strEqSpin_str = str(bForceEqSpin_star)
    strRotPer_str = str(dRotPeriod_star)
    strRadGyr_str = str(dRadGyra_star)
    strTidalQ_str = str(dTidalQ_star[n])
    strdK2_str = str(dK2_star)
    saTidePerts_star = saTidePerts_star
    strsaOutputOrder = saOutputOrder_s
    
    #vpl.in
    sys_name = Sys_name
    iVerbose_str = str(iVerbose)
    bOverwrite_str = str(bOverwrite)
    saBodyFiles = saBodyFiles
    UnitMass = UnitMass
    UnitLength = UnitLength
    UnitTime = UnitTime
    UnitAngle = UnitAngle
    UnitTemp = UnitTemp
    bDoLog_str = str(bDoLog)
    iDigits_str = str(iDigits)
    dMinValue_str = str(dMinValue)
    bDoForward_str = str(bDoForward)
    bVarDt_str = str(bVarDt)
    dEta_str = str(dEta)
    dStopTime_str = str(int(dStopTime))
    dOutputTime_str = str(dOutputTime)
    
    
    b = open('./runs/'+name_idx+'/b.in','w')
    b_content = ('sName\t'+ name_b + 
                 '\nsaModules\t'+modules_b+
                 '\nsTideModel\t'+sTideModel_b+
                 '\n\ndMass\t'+mass_str_b+
                 '\ndRadius\t'+radius_str_b+
                 '\ndObliquity\t'+obl_str_b+
                 '\ndRadGyra\t'+radgy_str_b+
                 '\n\ndEcc\t'+ecc_str_b+
                 '\ndSemi\t'+semi_str_b+
                 '\n\ndInc\t'+dInc_str_b+
                 '\ndLongA\t'+dLongA_str_b+
                 '\ndArgP\t'+dArgP_str_b+
                 #'\nsOrbitModel\t'+sOrbitModel_b+  #for Distorb
                 #'\nbOutputLapl\t'+bOutputLapl_str_b+ #for Distorb
                 '\n\ndPrecA\t'+dPrecA_str_b+     #distrot? 
                 '\nbCalcDynEllip\t'+bCalcDynEllip_str_b+ #distrot?
                 '\n\nbForceEqSpin\t'+eqspin_str_b+
                 '\ndTidalQ\t'+q_str_b+
                 '\ndK2\t'+dK2_str_b+
                 '\nsaTidePerts\t'+perts_str_b+
                 '\n\nsaOutputOrder\t'+outorder_str_b+'\n')
    b.write(b_content)
    b.close()
    
    c = open('./runs/'+name_idx+'/c.in','w')
    c_content = ('sName\t'+ name_c + 
                 '\nsaModules\t'+modules_c+
                 '\n\ndMass\t'+mass_str_c+
                 '\ndRadius\t'+radius_str_c+
                 '\ndObliquity\t'+obl_str_c+
                 '\ndRadGyra\t'+radgy_str_c+
                 '\n\ndEcc\t'+ecc_str_c+
                 '\ndSemi\t'+semi_str_c+
                 '\n\ndInc\t'+dInc_str_c+
                 '\ndLongA\t'+dLongA_str_c+
                 '\ndArgP\t'+dArgP_str_c+
                 '\n\nbForceEqSpin\t'+eqspin_str_c+
                 '\ndTidalQ\t'+q_str_c+
                 '\ndK2\t'+dK2_str_c+
                 '\nsaTidePerts\t'+perts_str_c+
                 '\n\nsaOutputOrder\t'+outorder_str_c+'\n')
    c.write(c_content)
    c.close()
    
    star = open('./runs/'+name_idx+'/star.in','w')
    star_content = ('sName\t'+name_star+
                    '\nsaModules\t'+modules_star+
                    '\n\ndMass\t'+strMass_str+
                    '\ndRadius\t'+strRad_str+
                    '\ndObliquity\t'+strObl_str+
                    '\ndRotPeriod\t'+strRotPer_str+
                    '\ndRadGyra\t'+strRadGyr_str+
                    '\n\ndTidalQ\t'+strTidalQ_str+
                    '\ndK2\t'+strdK2_str+
                    '\n\nsaTidePerts\t'+saTidePerts_star+
                    '\n\nsaOutputOrder\t'+strsaOutputOrder+'\n')
    star.write(star_content)
    star.close()
    
    vpl = open('./runs/'+name_idx+'/vpl.in','w')
    vpl_content = ('sSystemName\t'+sys_name+
                   '\niVerbose\t'+iVerbose_str+
                   '\nbOverwrite\t'+bOverwrite_str+
                   '\n\nsaBodyFiles\t'+saBodyFiles+
                   '\n\nsUnitMass\t'+UnitMass+
                   '\nsUnitLength\t'+UnitLength+
                   '\nsUnitTime\t'+UnitTime+
                   '\nsUnitAngle\t'+UnitAngle+
                   '\nsUnitTemp\t'+UnitTemp+
                   '\n\nbDoLog\t'+bDoLog_str+
                   '\niDigits\t'+iDigits_str+
                   '\ndMinValue\t'+dMinValue_str+
                   '\n\nbDoForward\t'+bDoForward_str+
                   '\nbVarDt\t'+bVarDt_str+
                   '\ndEta\t'+dEta_str+
                   '\ndStopTime\t'+dStopTime_str+
                   '\ndOutputTime\t'+ dOutputTime_str+'\n')
    vpl.write(vpl_content)
    vpl.close()
    
    
        
 
    n += 1







print ('IC files generated')

IC files generated


In [7]:
#runs each simulation
t= 10000
s = 0
# can i add in 

while s <= t-1:
    name_idx = '%05i'%s
    folder_name = name_idx
    working_directory = './lehmer_reprod/'+folder_name
    p = subprocess.call(['vplanet vpl.in > log'], shell=True, cwd = working_directory)
    print(name_idx)
    s += 1
    continue



print('Runs Completed')

00000
00001
00002
00003
00004
00005
00006
00007
00008
00009
00010
00011


KeyboardInterrupt: 